In [19]:
%mkdir dataset

mkdir: cannot create directory ‘dataset’: File exists


# load datasets


In [20]:
import pandas as pd
#loading the id and the LABELS
sentiment_pd = pd.read_csv("/content/dataset/sentiment_labels.txt",
                  sep="|",
                  header=0,
                  names=['phrase_id', 's_label'],
                  on_bad_lines='skip')

#loading the id and the PHRASES
phrases_pd = pd.read_csv("/content/dataset/dictionary.txt",
                         sep="|",
                         header=None,
                         on_bad_lines='skip',
                         names=['phrase', 'phrase_id'])
v = pd.read_csv("/content/dataset/dictionary.txt", sep="|", header=0)

In [21]:
comments = pd.read_csv("/content/dataset/Reddit_Data.csv")


In [22]:
# a = phrases_pd[phrases_pd['phrase_id'] == sentiment_pd['phrase_id'].loc[sentiment_pd['s_label'].idxmax()]]
# # print(list(a['phrase']))
comments

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1
...,...,...
37244,jesus,0
37245,kya bhai pure saal chutiya banaya modi aur jab...,1
37246,downvote karna tha par upvote hogaya,0
37247,haha nice,1


#Discretizzare labels

In [23]:
categories = { 0.34:'negative', 0.65:'neutral', 1:'positive'}
categories_comments = { -1 :'negative', 0:'neutral', 1:'positive'}
thresholds = list(categories.keys())
thresholds_c = list(categories_comments.keys())

sentiment_pd['s_label_disc'] = sentiment_pd['s_label'].apply(lambda x: next(categories[t] for t in thresholds if x <= t), 'OUTLIER')

comments['category_disc'] = comments['category'].apply(lambda x: next(categories[t] for t in thresholds if x<=t))

In [24]:
reviews = pd.DataFrame({'phrase_id': sentiment_pd['phrase_id'], 'phrase': phrases_pd['phrase'], 's_label_disc':sentiment_pd['s_label_disc'],  's_label': sentiment_pd['s_label']})
reviews['phrase'].loc[reviews['s_label'].idxmax()]

"'s not Little Nicky . And for many of us , that 's good enough"

# **data cleaning**

#EMPTY DATA cleaning

In [25]:
#Remove NON STRING ELEMENTS and EMPTY STRINGS
comments = comments[comments['clean_comment'].apply(lambda x: isinstance(x,str) and len(x) > 0)]
#lower strings
comments['clean_comment'] = comments['clean_comment'].apply(lambda x: x.lower())

#Remove NON STRING ELEMENTS and EMPTY STRINGS
reviews = reviews[reviews['phrase'].apply(lambda x: isinstance(x,str) and len(x) > 0)]
#lower strings
reviews['phrase'] = reviews['phrase'].apply(lambda x: x.lower())

<ipython-input-25-d15c87118f19>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['clean_comment'] = comments['clean_comment'].apply(lambda x: x.lower())
<ipython-input-25-d15c87118f19>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews['phrase'] = reviews['phrase'].apply(lambda x: x.lower())


# removing stop words

In [26]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = list(stopwords.words('english'))
#stop_words_df = pd.DataFrame(data=stop_words, columns=['STOP_WORD'])
#Removing the stopwords in x
comments['clean_comment'] = comments['clean_comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

#Removing the stopwords in x
reviews['phrase'] = reviews['phrase'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-26-087ab38e6ee0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['clean_comment'] = comments['clean_comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))


In [27]:
#punctuation removal nuetral
import string
comments['clean_comment'] = comments['clean_comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in string.punctuation]))

#remove empty raws
comments = comments[comments['clean_comment'].apply(lambda x: isinstance(x,str) and len(x) > 0)]

#punctuation removal nuetral
import string
reviews['phrase'] = reviews['phrase'].apply(lambda x: ' '.join([word for word in x.split() if word not in string.punctuation]))

#remove empty raws
reviews = reviews[reviews['phrase'].apply(lambda x: isinstance(x,str) and len(x) > 0)]

#neutral is cleaned

<ipython-input-27-74d079039f6f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['clean_comment'] = comments['clean_comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in string.punctuation]))


In [28]:
import nltk
from nltk.corpus import stopwords

# Download NLTK resources if not already installed
nltk.download('punkt')
nltk.download('stopwords')

def remove_punctuation_stopwords(text):
  # Tokenize the text
  tokens = nltk.word_tokenize(text)
  # Filter out punctuation and stopwords
  stop_words = set(stopwords.words('english'))  # Download stopwords for your language
  filtered_tokens = [word.lower() for word in tokens if word.isalnum() and word not in stop_words]
  return ' '.join(filtered_tokens)

reviews['phrase'] = reviews['phrase'].apply(remove_punctuation_stopwords)
comments['clean_comment'] = comments['clean_comment'].apply(remove_punctuation_stopwords)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
reviews['phrase'] = reviews['phrase'].apply(lambda x: ' '.join([char for char in x.split() if not char.isdigit()]))

#remove empty raws
reviews = reviews[reviews['phrase'].apply(lambda x: isinstance(x,str) and len(x) > 0)]

comments['clean_comment'] = comments['clean_comment'].apply(lambda x: ' '.join([char for char in x.split() if not char.isdigit()]))

#remove empty raws
comments = comments[comments['clean_comment'].apply(lambda x: isinstance(x,str) and len(x) > 0)]

# **Learning**

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Assuming you have two datasets: dataset1 and dataset2
# dataset1 and dataset2 should have similar data structure

# Split dataset1 into training and testing sets
x_train_reviews, x_test_reviews, y_train_reviews, y_test_reviews = train_test_split( reviews['phrase'],
                                                                                     reviews['s_label_disc'],
                                                                                     test_size=0.2,
                                                                                     random_state=42)

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(x_train_reviews)

# Initialize logistic regression classifier
classifier = LogisticRegression()

# Train the classifier on dataset1
classifier.fit(X_train_tfidf, y_train_reviews)

# Evaluate the classifier on dataset2
X_test_tfidf_reviews = tfidf_vectorizer.transform(x_test_reviews)
y_pred_reviews = classifier.predict(X_test_tfidf_reviews)

acc_rev = accuracy_score(y_test_reviews, y_pred_reviews)
print("Accuracy for Reviews: {:0.3f}".format(acc_rev))


clas = classifier
# Fine-tune the classifier on dataset2

# Split dataset2 into training and testing sets
X_train_dataset2, X_test_comments, y_train_comments, y_test_comments = train_test_split(comments['clean_comment'],
                                                                                        comments['category_disc'],
                                                                                        test_size=0.2,
                                                                                        random_state=42)

# Transform the training data using the same TF-IDF vectorizer fitted on dataset1
X_train_tfidf_comments = tfidf_vectorizer.transform(X_train_dataset2)

# Retrain the classifier on dataset2
classifier.fit(X_train_tfidf_comments, y_train_comments)

# Evaluate the classifier on dataset2
X_test_tfidf_comments = tfidf_vectorizer.transform(X_test_comments)
y_pred_comments = classifier.predict(X_test_tfidf_comments)

# Print classification report
print("Classification Report for Dataset2:")
print(classification_report(y_test_comments, y_pred_comments))


# Print Accuracy

acc = accuracy_score(y_test_comments, y_pred_comments)
print("Accuracy for Comments: {:0.3f}".format(acc))


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy for Reviews: 0.615
Classification Report for Dataset2:
              precision    recall  f1-score   support

    negative       0.84      0.93      0.88      4173
    positive       0.89      0.76      0.82      3202

    accuracy                           0.86      7375
   macro avg       0.86      0.85      0.85      7375
weighted avg       0.86      0.86      0.85      7375

Accuracy for Comments: 0.856


In [33]:
print("Accuracy for Comments: {:0.3f}".format(acc))

#Testing
test_phrases1 = ['test phrasing is good ', 'i don wanna go', 'that is some big Bullshit man']
test_phrases = [
    "i gotta have sex tonight",
    "I love this product, it's amazing!",
    "This movie was terrible, I wasted my money.",
    "The service at the restaurant was excellent, I highly recommend it.",
    "The book was boring and poorly written.",
    "The weather today is perfect for a picnic.",
    "I'm so excited to try out the new features in the latest software update.",
    "The customer support was rude and unhelpful.",
    "I feel so grateful for all the support and encouragement I've received.",
    "The performance of the stock market has been volatile lately.",
    "The quality of the food at the cafe was subpar, I won't be going back."
]

pd.DataFrame({'LABEL': classifier.predict(tfidf_vectorizer.transform(test_phrases1)), 'FRASE': test_phrases1})
# pd.DataFrame({'LABEL': clas.predict(tfidf_vectorizer.transform(test_phrases1)), 'FRASE': test_phrases1})

Accuracy for Comments: 0.856


,LABEL,FRASE
0,positive,test phrasing is good
1,negative,i don wanna go
2,negative,that is some big Bullshit man


# **-------------------------**

In [32]:
c = reviews.loc[15:250]
c

,phrase_id,phrase,s_label_disc,s_label
15,15,wow,positive,0.81944
16,16,zoom,positive,0.76389
29,29,million,neutral,0.48611
30,30,million charmer,neutral,0.52778
32,32,million,negative,0.29167
...,...,...,...,...
243,243,image,negative,0.33333
244,244,astonishing,neutral,0.47222
248,248,lilo stitch edgy piece disney animation hit si...,negative,0.33333
249,249,lilo stitch reach emotion timelessness disney ...,neutral,0.48611
